In [11]:
import pandas as pd
import random
import numpy as np

In [12]:
def levenshtein_distance(s1, s2):
    """Compute the Levenshtein distance between two strings."""
    if len(s1) < len(s2):
        return levenshtein_distance(s2, s1)

    if len(s2) == 0:
        return len(s1)

    previous_row = range(len(s2) + 1)
    for i, c1 in enumerate(s1):
        current_row = [i + 1]
        for j, c2 in enumerate(s2):
            insertions = previous_row[j + 1] + 1
            deletions = current_row[j] + 1
            substitutions = previous_row[j] + (c1 != c2)
            current_row.append(min(insertions, deletions, substitutions))
        previous_row = current_row

    return previous_row[-1]

In [13]:
def calculate_probability(distance, max_len):
    """Calculate probability of match based on Levenshtein distance and maximum length of strings."""
    if max_len == 0:
        return 1
    return 1 - distance / max_len

In [14]:
def prob_con(df1, df2, columns_df1, columns_df2, noise_level=0.01):
    results = []
    # Precompute maximum lengths for normalization
    max_lengths = {col1: max(df1[col1].astype(str).apply(len).max(), df2[col2].astype(str).apply(len).max())
                   for col1, col2 in zip(columns_df1, columns_df2)}

    # Iterate over each record in df1
    for index1, row1 in df1.iterrows():
        best_match = None
        max_score = 0
        # Compare with each record in df2
        for index2, row2 in df2.iterrows():
            total_prob = 1
            # Calculate combined probability over all matching columns
            for col1, col2 in zip(columns_df1, columns_df2):
                str1 = str(row1[col1])
                str2 = str(row2[col2])
                distance = levenshtein_distance(str1, str2)
                prob = calculate_probability(distance, max_lengths[col1])
                total_prob *= prob
            # Add noise to the total probability
            noisy_prob = total_prob + random.uniform(0, noise_level)
            # Update the best match if the current noisy probability is higher
            if noisy_prob > max_score:
                best_match = index2
                max_score = noisy_prob
        # Append the best match and the max score for this record
        results.append({'df1_index': index1, 'df2_index': best_match, 'score': max_score})
    return pd.DataFrame(results)


In [15]:
def convert_numbers_to_binary(df, column_name):
    """Check if a feature in the DataFrame is numeric and convert each number to its binary representation."""
    # Check if the column is numeric
    if pd.api.types.is_numeric_dtype(df[column_name]):
        # Convert each number to binary (without the '0b' prefix)
        df[column_name] = df[column_name].apply(lambda x: format(int(x), 'b'))
    else:
        print(f"The column '{column_name}' is not numeric and cannot be converted to binary.")
    
    return df

In [16]:
# Generate synthetic data directly for df1
df1 = pd.DataFrame({
    'age': np.random.randint(20, 70, size=100),  # Random ages between 20 and 70
    'gender': np.random.randint(0, 2, size=100),  # Random binary gender
    'education_level': np.random.randint(1, 5, size=100),  # Random education level from 1 to 4
    'income': np.random.randint(20000, 100000, size=100),  # Random income between 20,000 and 100,000
    'height': np.random.randint(150, 200, size=100)  # Random height between 150 cm and 200 cm
})

# Generate independent synthetic data directly for df2 with only 20 samples
df2 = pd.DataFrame({
    'years': np.random.randint(20, 70, size=20),  # Similar age range for easy comparison
    'sex': np.random.randint(0, 2, size=20),  # Random binary gender
    'edu_level': np.random.randint(1, 5, size=20),  # Random education level from 1 to 4
    'library_access': np.random.randint(0, 2, size=20),  # Binary, 1 or 0
    'family_members': np.random.randint(1, 7, size=20)  # Family members from 1 to 6
})

# Print column names to verify
print("df1 columns:", df1.columns)
print("df2 columns:", df2.columns)

df1 columns: Index(['age', 'gender', 'education_level', 'income', 'height'], dtype='object')
df2 columns: Index(['years', 'sex', 'edu_level', 'library_access', 'family_members'], dtype='object')


In [17]:
# Create copies of original data before converting to binary for merging purposes
original_df1 = df1.copy()
original_df2 = df2.copy()

# Data Preparation - Define the column names to be matched
columns_df1 = ['age', 'gender', 'education_level']
columns_df2 = ['years', 'sex', 'edu_level']

# Convert all matching numeric columns to binary representation
for col_df1, col_df2 in zip(columns_df1, columns_df2):
    df1 = convert_numbers_to_binary(df1, col_df1)
    df2 = convert_numbers_to_binary(df2, col_df2)

In [18]:
# Run the probablistic concatenation
matches = prob_con(df1, df2, columns_df1, columns_df2)
matches.head(), matches['score'].describe()

(   df1_index  df2_index     score
 0          0         13  0.866880
 1          1         10  0.722151
 2          2          5  1.005730
 3          3         13  0.720618
 4          4          2  0.577830,
 count    100.000000
 mean       0.757224
 std        0.117573
 min        0.431027
 25%        0.715941
 50%        0.722081
 75%        0.863473
 max        1.008957
 Name: score, dtype: float64)

In [19]:
# Define columns from df1 and df2 that are not matched
unmatched_df1_columns = [col for col in original_df1.columns if col not in columns_df1]
unmatched_df2_columns = [col for col in original_df2.columns if col not in columns_df2]

# Initialize a list to collect rows for merged data
merged_rows = []

# Iterate over the matches to combine rows from original df1 and df2 using the original data
for _, match in matches.iterrows():
    df1_row = original_df1.loc[match['df1_index'], unmatched_df1_columns + columns_df1]
    df2_row = original_df2.loc[match['df2_index'], unmatched_df2_columns]
    combined_row = pd.concat([df1_row, df2_row], axis=0)
    merged_rows.append(combined_row)

# Convert the list of Series into a DataFrame and transpose
merged_data = pd.concat(merged_rows, axis=1).T
merged_data.columns = unmatched_df1_columns + columns_df1 + unmatched_df2_columns


In [20]:
# Print the merged DataFrame
merged_data

,income,height,age,gender,education_level,library_access,family_members
0,24942,179,59,1,2,1,2
1,55463,185,63,0,3,1,4
2,78983,194,51,1,4,0,2
3,94130,166,45,1,2,1,2
4,79599,150,51,0,1,0,6
...,...,...,...,...,...,...,...
95,61288,154,56,0,3,0,3
96,22039,164,27,0,2,0,4
97,48114,177,57,0,2,0,6
98,59128,169,46,1,3,1,5
